In [1]:
import requests
from bs4 import BeautifulSoup
import time

## 預計抓取資料
- 飯店名稱
- 房客評價平均星等
- 房客評論數
- 房客留言內容
- 參考價格(爬蟲爬得時候的價格)
- 相關訂房網站的價格
- 飯店地址
- 飯店ranking (tripadviosr給的)

---
可能可以抓的：
- 飯店設備(游泳池之類)
- 是否有供餐
- 是否提供wifi
- 是否有車位
- 有沒有room service
- 訂房後能不能免費取消
- 是否入住後才需要付款(不用網站上給訂金)

### 取飯店列表

In [13]:
url = 'https://www.tripadvisor.com.tw/Hotels-g293913-Taipei-Hotels.html'
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}

def get_hotel(url):
    #每次调用等待两秒, 避免被ban ip
    time.sleep(2)
    
    uri_list = []
    
    resp = requests.get(url)

    soup = BeautifulSoup(resp.text,'html.parser')

    #imgs = soup.select('div.aspect.is-hidden-tablet > div.inner')
    titles = soup.select('div.listing_title > a[target="_blank"]')
    paimings = soup.select('div.popindex') # 排名
    prices = soup.select('div[data-sizegroup="mini-meta-price"]')
    agoda = soup.select('div[title="Agoda.com"]')
    hotel_com = soup.select('div[title="Hotels.com"]')
    
    # 取其他平台的價格
#     print(agoda[0].find_next_siblings('div')[0].get_text())
#     print(hotel_com[0].find_next_siblings('div'))

    for title,paiming,price,ago in zip(titles,paimings,prices, agoda):
        
        data = {
            'title':title.get_text(),
            'paiming':paiming.get_text(),
            'recommend_price':price.get_text(), # tripadviosr推薦的訂房網站價格，不一定最低
            'agoda_price': ago.find_next_siblings('div')[0].get_text()
#             'uri': 'https://www.tripadvisor.com.tw' + title.get('href')
        }
        print(data)
        
        uri_list.append('https://www.tripadvisor.com.tw' + title.get('href'))
    
    return uri_list

In [11]:
a = get_hotel(url)

NT$2,996
[<div class="price autoResize" data-clickpart="text_price" data-index="3">NT$3,124</div>]


單一家飯店

In [111]:
all_uri = get_hotel(url)

{'title': '台北凱達大飯店', 'paiming': '台北排名第 1 的超值住宿地點，共 1,200 家', 'price': 'NT$6,086'}
{'title': '新驛旅店 - 復興北路店', 'paiming': '台北排名第 2 的超值住宿地點，共 1,200 家', 'price': 'NT$3,714'}
{'title': '路境行旅', 'paiming': '台北排名第 3 的超值住宿地點，共 1,200 家', 'price': 'NT$1,977'}
{'title': '圓山大飯店', 'paiming': '台北排名第 4 的超值住宿地點，共 1,200 家', 'price': 'NT$1,934'}
{'title': '丰居旅店雙連館', 'paiming': '台北排名第 5 的超值住宿地點，共 1,200 家', 'price': 'NT$4,819'}
{'title': '台北美侖大飯店', 'paiming': '台北排名第 6 的超值住宿地點，共 1,200 家', 'price': 'NT$1,628'}
{'title': '儷夏商旅 - 西門捷運館', 'paiming': '台北排名第 7 的超值住宿地點，共 1,200 家', 'price': 'NT$3,009'}
{'title': '新驛旅店- 台北車站一館', 'paiming': '台北排名第 8 的超值住宿地點，共 1,200 家', 'price': 'NT$2,711'}
{'title': '新尚旅店', 'paiming': '台北排名第 9 的超值住宿地點，共 1,200 家', 'price': 'NT$1,539'}
{'title': '薆悅酒店', 'paiming': '台北排名第 10 的超值住宿地點，共 1,200 家', 'price': 'NT$1,908'}
{'title': '倆人旅店', 'paiming': '台北排名第 11 的超值住宿地點，共 1,200 家', 'price': 'NT$1,616'}
{'title': '老爺會館台北林森', 'paiming': '台北排名第 12 的超值住宿地點，共 1,200 家', 'price': 'NT$2,320'}
{'title': '

自動換頁(去抓其他飯店)

In [15]:
url_list = ['https://www.tripadvisor.com.tw/Hotels-g293913-oa{}-Taipei-Hotels.html'.format(str(i)) for i in range(0,1200,30)]

In [16]:
hotel_url = get_hotel(url_list[0])
hotel_url = get_hotel(url_list[1])

{'title': '台北凱達大飯店', 'paiming': '台北排名第 1 的超值住宿地點，共 1,200 家', 'lowest_price': 'NT$5,900', 'agoda_price': 'NT$3,095'}
{'title': '新驛旅店 - 復興北路店', 'paiming': '台北排名第 2 的超值住宿地點，共 1,200 家', 'lowest_price': 'NT$3,095', 'agoda_price': 'NT$2,589'}
{'title': '路境行旅', 'paiming': '台北排名第 3 的超值住宿地點，共 1,200 家', 'lowest_price': 'NT$1,977', 'agoda_price': 'NT$2,034'}
{'title': '圓山大飯店', 'paiming': '台北排名第 4 的超值住宿地點，共 1,200 家', 'lowest_price': 'NT$1,936', 'agoda_price': 'NT$4,819'}
{'title': '台北美侖大飯店', 'paiming': '台北排名第 5 的超值住宿地點，共 1,200 家', 'lowest_price': 'NT$4,819', 'agoda_price': 'NT$2,311'}
{'title': '儷夏商旅 - 西門捷運館', 'paiming': '台北排名第 6 的超值住宿地點，共 1,200 家', 'lowest_price': 'NT$2,709', 'agoda_price': 'NT$1,855'}
{'title': '新驛旅店- 台北車站一館', 'paiming': '台北排名第 7 的超值住宿地點，共 1,200 家', 'lowest_price': 'NT$2,304', 'agoda_price': 'NT$2,334'}
{'title': '新尚旅店', 'paiming': '台北排名第 8 的超值住宿地點，共 1,200 家', 'lowest_price': 'NT$1,538', 'agoda_price': 'NT$2,427'}
{'title': '薆悅酒店', 'paiming': '台北排名第 9 的超值住宿地點，共 1,200 家', 'lowest

房間售完/ 已被網站排除推薦

In [18]:
hotel_url = get_hotel(url_list[38])

### 根據各別飯店取細節

In [85]:
import json

def get_hotel_detail(url):
    time.sleep(2)
    
    resp = requests.get(url)
    
    soup = BeautifulSoup(resp.text,'html.parser')
    
    meta_data = soup.select('script[type="application/ld+json"]')
    
    # 使用者評論內容
    user_comment = soup.select('.common-text-ReadMore__content--2X4LR')
#     user_comment = soup.findAll("div", {"class": "common-text-ReadMore__content--2X4LR"})
#     print(user_comment[0].get_text())
#     print(user_comment[0].text)
    print(user_comment[0].q.span)
    
#     for i in meta_data:
#         print(i.get_text())

    return meta_data[0].get_text()

In [86]:
json.loads(get_hotel_detail(all_uri[0]))

<span>每年都會固定買旅展餐卷，這裡就像第二個家，菜色真的進步很多，每年來都有驚喜，甜點區，現榨果汁，牛排等等，都很不錯，希望能繼續保持。</span>


{'@context': 'http://schema.org',
 '@type': 'LodgingBusiness',
 'name': '台北花園大酒店',
 'url': '/Hotel_Review-g13808853-d1224203-Reviews-Taipei_Garden_Hotel-Zhongzheng_District_Taipei.html',
 'image': 'https://media-cdn.tripadvisor.com/media/photo-s/09/32/1f/6c/caption.jpg',
 'priceRange': 'NT$3,146 - NT$6,639 (根據標準客房的平均房價)',
 'aggregateRating': {'@type': 'AggregateRating',
  'ratingValue': '4.5',
  'reviewCount': '2750'},
 'address': {'@type': 'PostalAddress',
  'streetAddress': '中正區中華路二段1號',
  'addressLocality': '',
  'addressRegion': '',
  'postalCode': '10065',
  'addressCountry': {'@type': 'Country', 'name': '台灣'}}}